## Query 2

> ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado 'Refunded'? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

estas direcciones = 5 códigos postales más comunes

Los pasos en este caso serían:

1. Importar el dataset de ordenes crudas (.data/raw/orders.csv) a un RDD.
2. Filtar aquellas con datos invalidos o que no esten en estado 'Refunded'.
3. Mapear el código postal de la dirección (shipping en este caso).
4. Filtrar los registros que no correspondan con un código postal válido.
5. Agrupar por código postal y contar la cantidad de ocurrencias.
6. Tomar los 5 códigos postales más comunes.

Luego para el nombre más frecuente:

1. Importar el dataset de clientes (.data/raw/customers.csv) a un RDD.
2. Filtrar aquellos con datos invalidos.
3. Mapear el nombre y el código postal.
4. Filtrar los registros que no correspondan con un código postal válido.
5. Filtrar aquellos cuyo código postal esté entre los 5 más comunes obtenidos anteriormente.
6. Agrupar por nombre y contar la cantidad de ocurrencias.
7. Tomar el nombre más frecuente.

In [1]:
import re
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("RefundedOrders").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [3]:
df_orders = spark.read.csv("../../.data/raw/orders.csv", header=True, inferSchema=True)
rdd_orders = df_orders.rdd


def extract_zip(address):
    if not address:
        return None
    match = re.search(r"(\d{5})$", address)
    return match.group(1) if match else None


REFUND_STATUS = "REFUNDED"

rdd_refunded_orders = rdd_orders.map(
    lambda r: {
        "status": r.status.strip().upper() if getattr(r, "status", None) else None,
        "shipping_address": (
            r.shipping_address if getattr(r, "shipping_address", None) else None
        ),
    }
).filter(
    lambda row: row["status"] == REFUND_STATUS and row["shipping_address"] is not None
)


refunded_with_zip = rdd_refunded_orders.map(
    lambda row: extract_zip(row["shipping_address"])
)

zip_counts = (
    refunded_with_zip.filter(lambda z: z is not None)
    .map(lambda z: (z, 1))
    .reduceByKey(lambda a, b: a + b)
)

In [4]:
top_5_zips = zip_counts.takeOrdered(5, key=lambda x: -x[1])

print("Top 5 códigos postales más comunes para órdenes refunded:")
for zip_code, count in top_5_zips:
    print(f"{zip_code}: {count}")

top_zips_set = set(zip for zip, _ in top_5_zips)

# Para usar en la query de customers
broadcast_top_zips = spark.sparkContext.broadcast(top_zips_set)

Top 5 códigos postales más comunes para órdenes refunded:
70696: 6
47612: 5
11954: 5
83755: 5
59883: 5


In [5]:
df_customers = spark.read.csv(
    "../../.data/raw/customers.csv", header=True, inferSchema=True
)
rdd_customers = df_customers.rdd

rdd_customers_min = rdd_customers.map(
    lambda r: {
        "address": r.address if getattr(r, "address", None) else None,
        "first_name": r.first_name.strip() if getattr(r, "first_name", None) else None,
    }
)

In [6]:
customers_filtered = (
    rdd_customers_min.filter(
        lambda row: row["address"] is not None and row["first_name"] is not None
    )
    .map(
        lambda row: {
            "zip": extract_zip(row["address"]),
            "first_name": row["first_name"],
        }
    )
    .filter(lambda x: x["zip"] is not None and x["zip"] in broadcast_top_zips.value)
)

name_counts = customers_filtered.map(lambda x: (x["first_name"], 1)).reduceByKey(
    lambda a, b: a + b
)

if name_counts.isEmpty():
    most_frequent_name = None
else:
    most_frequent_name = name_counts.reduce(lambda a, b: a if a[1] > b[1] else b)

print(
    f"Nombre más frecuente entre los clientes de esos códigos postales: {most_frequent_name}"
)

Nombre más frecuente entre los clientes de esos códigos postales: ('Amy', 1)
